In [ ]:
## Simulationen laufen lassen und auswerten

In [ ]:
# jeweils daten simulieren und vorauswerten; plots erstellen gehören in plots

# funktionen für:
# - parameter scan, beinhaltet Summe
# - 
# - Monte Carlo: Nein, zu viel Zeit, außer jemand hat besondere Langeweile

## Analysis File

For the functionality in this file are following libraries: <b>numpy, pickle, itertools, typing, collections </b> and <b>functools</b> required. Also self implemented functions from <b> rk4</b> and <b> awareness</b> are needed. 

This file is used to run and evaluate the all of the simulations. This includes simualtion for <b> rungekutta4, default SIR</b> and <b> informormation</b>. For this data has to be loaded from the saved files. The results of the simulations are saved in additional files. The files are stored in the <b> data</b>-folder. They are named with the corresponding function names and parameters. <br>
Detaild aspects and code-design decisions are being explained in the functions itself.


In [ ]:
import numpy as np
import pickle
from itertools import product
from typing import Union
from collections.abc import Callable
from functools import partial
import import_ipynb

In [ ]:
from rk4 import rk4

importing Jupyter notebook from rk4.ipynb


In [ ]:
from awareness import awareness_SIR, g2

importing Jupyter notebook from awareness.ipynb


In [ ]:
def awareness_param_names():
    return ['alpha', 'omega', 'lam', 'rho', 'kappa']

In [ ]:
def simulate_rk4(model: Callable[[float, np.ndarray], np.ndarray], y0: np.ndarray, params: dict, tmax: float, dt: float=0.1, fname='test_data.txt', save=True)->None:
    n = int(tmax/dt)
    print(params)
    p_model = partial(model, **params)
    t, y = rk4(p_model, y0, 0, tmax, n)
    data = (t, y)
    if save:
        save_data(data, fname)
    return t, y

In [ ]:
def check_probability_conserved(y: np.ndarray)->None:
    print(f'probability conserved: {np.array([np.sum(y[i, j], axis=(0, 1)).all() for i in range(y.shape[0]) for j in range(y.shape[1])]).all()}')

In [ ]:
def axis_product(params: list, default_params: list)->tuple:
    """
    returnes combinations for parameters, varying one axis at a time
    params: list containing lists for different parameter values
    default_params: default values to use when another axis is being varied
    """
    combinations = []
    combinations.append(default_params)
    for i in range(len(default_params)):
        options = [params[j] if j == i else [default_params[j]] for j in range(len(default_params))]
        combinations += [x for x in product(*options)]
    combinations_filtered = []
    for comb in combinations:
        if comb not in combinations_filtered:
            combinations_filtered.append(comb)
    return combinations_filtered


In [ ]:
def str_name(param_names: list, params: list)->str:
    s = '___'.join([f'{param_names[i]}_{params[i]:.2e}' for i in range(len(params))]) + '.pkl'
    return s


In [ ]:
def save_data(data, fname):
    with open(fname, "wb+") as f:
        pickle.dump(data, f)

In [ ]:
def load_data(fname):
    with open(fname, 'rb') as f:
        data = pickle.load(f)
    return data

In [ ]:
def run_simulation(infection_params, awareness_params, simulation_params, name):
    beta0, gamma0 = infection_params
    alpha0, omega0, lam0, rho0, kappa0 = awareness_params
    grid_size, info_compartments, tmax, dt = simulation_params
    
    y0 = np.zeros(shape=(grid_size, grid_size, 3, info_compartments))  # start parameters
    y0[:,:,0,-1] = 1  # everyone is susceptible, no awareness at the start

    mid = int((grid_size-1)/2)
    y0[mid, mid, 1, -1] = 1  # one infection in the middle
    y0[mid, mid, 0, -1] = 0  # remove infected from susceptible pool

    model = partial(awareness_SIR, beta=beta0, gamma=gamma0)

    # simulating

    params_dict = dict(zip(awareness_param_names(), awareness_params))
    filename = f'data/{name}_simulation___.' + str_name(awareness_param_names(), awareness_params)
    simulate_rk4(model, y0, params_dict, tmax, dt, fname=filename)

    # analyze

    t, y = load_data(filename)
    data_av = np.average(y[:, :, :, :], axis=(0, 1)) # average over grid
    awareness_level = g2(rho0, data_av[0])  # everyone is susceptible, there are no infections
    save_data((t, awareness_level), f'data/{name}_simulation_awareness.pkl')


In [ ]:
# TODO: restructure so data does not have to be saved (LIMITED SPACE AVAILABLE)

def run_sensitivity_sweep(infection_params, awareness_params, awareness_params_variation, simulation_params):

    beta0, gamma0 = infection_params
    alpha0, omega0, lam0, rho0, kappa0 = awareness_params
    alpha, omega, lam, rho, kappa = awareness_params_variation
    grid_size, info_compartments, tmax, dt = simulation_params

    mid = int((grid_size-1)/2)
    y0 = np.zeros(shape=(grid_size, grid_size, 3, info_compartments))  # start parameters
    y0[:,:,0,-1] = 1  # everyone is susceptible, no awareness at the start
    y0[mid, mid, 1, -1] = 1  # one infection in the middle
    y0[mid, mid, 0, -1] = 0  # remove infected from susceptible pool

    model = partial(awareness_SIR, beta=beta0, gamma=gamma0)
    sweep_params = axis_product(awareness_params_variation, awareness_params)

    sim_results = {}  # store the main results from each simulation run (to conserve space)

    # simulate different scenarios

    for i, param_set in enumerate(sweep_params):
        print(f'{i+1}/{len(sweep_params)}:   ', end='')
        params_dict = dict(zip(awareness_param_names(), param_set))
        filename = 'data/' + str_name(awareness_param_names(), param_set)
        t, y = simulate_rk4(model, y0, params_dict, tmax, dt, fname=filename, save=False)
        infections = np.average(np.sum(y[:, :, 1, :], axis=2), axis=(0, 1))
        max_infections = np.max(infections)
        data = (param_set, [max_infections])
        sim_results[str(param_set)] = data

    # analyze

    # default params
    param_set = awareness_params
    #params_dict = dict(zip(awareness_param_names(), param_set))
    filename = 'data/' + str_name(awareness_param_names(), param_set)

    data = sim_results[str(param_set)]
    save_data(data, f'data/variation_default.pkl')

    # variations

    for i in range(len(awareness_params)):  # iterate over different parameters to vary
        options = [awareness_params_variation[j] if j == i else [awareness_params[j]] for j in range(len(awareness_params))]
        combinations = [x for x in product(*options)]

        param_variation = []
        concurrent_infections = []

        for param_set in combinations:
            filename = 'data/' + str_name(awareness_param_names(), param_set)

            params, max_infections = sim_results[str(param_set)]

            param_variation.append(param_set[i])
            concurrent_infections.append(max_infections[0])

        data = (param_variation, concurrent_infections)
        save_data(data, f'data/variation_{awareness_param_names()[i]}.pkl')

    print('done')



In [ ]:
def run_information_simulation(infection_params, awareness_params, simulation_params):
    beta0, gamma0 = infection_params
    alpha0, omega0, lam0, rho0, kappa0 = awareness_params
    grid_size, info_compartments, tmax, dt = simulation_params
    
    y0 = np.zeros(shape=(grid_size, grid_size, 3, info_compartments))  # start parameters
    y0[:,:,0,-1] = 1  # everyone is susceptible, no awareness at the start

    mid = int((grid_size-1)/2)
    y0[mid, mid, 0, 0] = 1  # one aware dot in the middle
    y0[mid, mid, 0, -1] = 0  # remove unaware from susceptible pool

    model = partial(awareness_SIR, beta=beta0, gamma=gamma0)

    # simulating

    params_dict = dict(zip(awareness_param_names(), awareness_params))
    filename = 'data/information_only___' + str_name(awareness_param_names(), awareness_params)
    simulate_rk4(model, y0, params_dict, tmax, dt, fname=filename)

    # analyze

    t, y = load_data(filename)
    data_av = np.average(y[:, :, :, :], axis=(0, 1)) # average over grid
    awareness_level = g2(rho0, data_av[0])  # everyone is susceptible, there are no infections
    save_data((t, awareness_level), 'data/awareness.pkl')


In [ ]:
if __name__ == '__main__':

    # model parameters

    # infections
    beta0  = 0.03  # virus transmission rate between two individuals
    gamma0 = 0.03  # recovery rate

    infection_params = [beta0, gamma0]

    # awareness
    alpha0 = 0.04  # awareness transmission
    omega0 = 0.04  # awareness creation
    lam0   = 0.04  # awareness fading over time
    rho0   = 0.6   # susceptibility reduction (rh0=0.9 means 90% reduction for first level, 81% reduction for second level etc.)
    kappa0 = 0.5   # self isolation (kappa=0.8 means 80% reduction through isolation)

    awareness_params = [alpha0, omega0, lam0, rho0, kappa0]

    # variation for sensitivity analysis
    k = 2
    kk = 2
    alpha = np.geomspace(0.5, 2, k)*alpha0
    omega = np.geomspace(0.5, 2, k)*omega0
    lam = np.geomspace(0.5, 2, k)*lam0
    rho = np.linspace(0.1, 0.9, kk)
    kappa = np.linspace(0, 0.8, kk)

    awareness_params_variation = [alpha, omega, lam, rho, kappa]

    # simulation parameters

    grid_size = 30  # length of square grid
    info_compartments = 5  # number of information compartments, the first one is complete reduction of
    # susceptibility for the known sick ones, the last one is for no awareness at all
    tmax = 50  # simulation time
    dt = 1  # time step

    simulation_params = [grid_size, info_compartments, tmax, dt]


    run_sensitivity_sweep(infection_params, awareness_params, awareness_params_variation, simulation_params)

1/11:   {'alpha': 0.04, 'omega': 0.04, 'lam': 0.04, 'rho': 0.6, 'kappa': 0.5}
2/11:   {'alpha': 0.02, 'omega': 0.04, 'lam': 0.04, 'rho': 0.6, 'kappa': 0.5}
3/11:   {'alpha': 0.08, 'omega': 0.04, 'lam': 0.04, 'rho': 0.6, 'kappa': 0.5}
4/11:   {'alpha': 0.04, 'omega': 0.02, 'lam': 0.04, 'rho': 0.6, 'kappa': 0.5}
5/11:   {'alpha': 0.04, 'omega': 0.08, 'lam': 0.04, 'rho': 0.6, 'kappa': 0.5}
6/11:   {'alpha': 0.04, 'omega': 0.04, 'lam': 0.02, 'rho': 0.6, 'kappa': 0.5}
7/11:   {'alpha': 0.04, 'omega': 0.04, 'lam': 0.08, 'rho': 0.6, 'kappa': 0.5}
8/11:   {'alpha': 0.04, 'omega': 0.04, 'lam': 0.04, 'rho': 0.1, 'kappa': 0.5}
9/11:   {'alpha': 0.04, 'omega': 0.04, 'lam': 0.04, 'rho': 0.9, 'kappa': 0.5}
10/11:   {'alpha': 0.04, 'omega': 0.04, 'lam': 0.04, 'rho': 0.6, 'kappa': 0.0}
11/11:   {'alpha': 0.04, 'omega': 0.04, 'lam': 0.04, 'rho': 0.6, 'kappa': 0.8}
done


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ac0f8ce2-3132-47be-a4d1-6216636e93ff' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>